In [2]:
import pandas as pd
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from getpass import getpass

In [ ]:
tweet_df = pd.read_csv()

In [23]:
fips_df = pd.read_csv('../res/big_data_tweets.csv', dtype='str')

In [25]:
fips_df = fips_df.loc[:, ['tweet_id', 'fips_county', 'fips_state']]

fips_df.head(10)

,tweet_id,fips_county,fips_state
0,1588320083335467009,NaN,05
1,1588320019259469824,NaN,NaN
2,1588320006840160256,NaN,54
3,1588319891446272001,NaN,26
4,1588319866385477632,NaN,NaN
5,1588319827651104769,09003,09
6,1588319810181529602,36047,36
7,1588319807375433728,06045,06
8,1588319706343124992,06053,06
9,1588319697808019456,NaN,NaN


In [ ]:
# df.to_sql(‘data’, con=conn, if_exists=’replace’, index=False)

# arguments are:

# name of the table
# connection
# if_exists : if the table already exists the function we want to apply . ex: ‘append’ help us add data instead of replacing the data.
# index : True or False

user = 'postgres'

password = getpass('Enter database password')

conn_string = f'postgres://{user}:{password}\
@gun-sentiment-db.ckbz1nmb8sin.us-west-2.rds.amazonaws.com/election_gun_sentiment_tweets_db'
  
db = create_engine(conn_string)
conn = db.connect()

  
# our dataframe
data = {'Name': ['Tom', 'dick', 'harry'],
        'Age': [22, 21, 24]}
  
# Create DataFrame
df = pd.DataFrame(data)
df.to_sql('data', con=conn, if_exists='replace',
          index=False)
conn = psycopg2.connect(conn_string
                        )
conn.autocommit = True
cursor = conn.cursor()
  
sql1 = '''select * from data limit 20;'''
cursor.execute(sql1)
for i in cursor.fetchall():
    print(i)

# conn.commit()
conn.close()